# LangGraph Chatbots with Memory

This notebook demonstrates how to build conversational chatbots using LangGraph with memory capabilities:

- **MessagesState**: Built-in state management for conversations
- **MemorySaver**: In-memory conversation persistence
- **Thread Management**: Separate conversation contexts
- **Streaming Responses**: Real-time conversation flow
- **Simple Architecture**: Minimal setup for maximum functionality

## Use Case: Basic Conversational AI

We'll build a chatbot that:
1. **Remembers Context**: Maintains conversation history
2. **Handles Multiple Users**: Uses thread IDs for separation
3. **Streams Responses**: Provides real-time interaction
4. **Persists Memory**: Keeps conversations across sessions

This pattern is essential for:
- **Customer Support Bots**: Need conversation context
- **Personal Assistants**: Remember user preferences and history
- **Educational Chatbots**: Track learning progress
- **Multi-user Applications**: Separate conversation contexts

## 1. Essential Imports

Let's import the core components needed for our memory-enabled chatbot:

In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
from IPython.display import display, Image
import os

## 2. State Definition with MessagesState

**MessagesState** is LangGraph's built-in state type for conversational applications.

### Key Features:
- **Automatic Message Handling**: Built-in support for conversation flow
- **Message Accumulation**: Uses `add_messages` to append new messages
- **Type Safety**: Ensures proper message structure
- **Memory Integration**: Works seamlessly with checkpointers

### Why Annotated with add_messages?
- **Accumulation Logic**: New messages are added to existing ones, not replaced
- **Conversation Flow**: Maintains the full conversation history
- **Memory Efficiency**: Handles message deduplication automatically

### Alternative Approaches:
- **Custom State**: More control but requires manual message handling
- **Simple List**: No automatic accumulation logic
- **MessagesState Inheritance**: Extend with additional fields

In [ ]:
class MyState(TypedDict):
    """State definition for our conversational chatbot.
    
    Uses Annotated with add_messages to ensure new messages
    are appended to the conversation history rather than
    replacing the entire message list.
    """
    messages: Annotated[list, add_messages]

## 3. LLM Setup

We'll configure our language model using AWS Bedrock with Claude.

### Environment Setup:
Make sure your `.env` file contains:
```
AWS_ACCESS_KEY_ID=your_access_key
AWS_SECRET_ACCESS_KEY=your_secret_key
```

### Model Selection:
- **Claude Sonnet**: Good balance of capability and speed
- **Claude Haiku**: Faster responses, lower cost
- **Claude Opus**: Highest capability for complex tasks

In [ ]:
from dotenv import load_dotenv
from langchain_aws import ChatBedrock

load_dotenv()

def get_chat_model():
    """Initialize AWS Bedrock ChatBedrock model.
    
    Returns:
        ChatBedrock: Configured LLM instance for conversations
    """
    llm = ChatBedrock(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
        aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
        region="us-east-1"
    )
    return llm

## 4. Conversation Chain

The **conversation chain** combines our prompt template with the LLM to create a reusable conversation component.

### Chain Components:
1. **System Prompt**: Defines the chatbot's personality and behavior
2. **MessagesPlaceholder**: Inserts the conversation history
3. **LLM**: Processes the prompt and generates responses

### Design Considerations:
- **Simple System Prompt**: Clear, concise instructions
- **Flexible Placeholder**: Handles variable-length conversations
- **Stateless Chain**: No internal state, relies on input messages

In [ ]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

def conversation_chain():
    """Create a conversation chain for chatbot responses.
    
    This chain:
    1. Takes the full conversation history
    2. Applies a system prompt for consistent behavior
    3. Generates contextually appropriate responses
    
    Returns:
        Chain that processes conversation messages
    """
    llm = get_chat_model()
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful chatbot that answers user queries. "
                  "Be conversational, friendly, and remember the context of our conversation. "
                  "If users share personal information, acknowledge and remember it."),
        MessagesPlaceholder(variable_name="messages")
    ])
    
    return prompt | llm

## 5. Conversation Node

The **conversation node** is our graph's processing unit that handles user messages.

### Node Responsibilities:
1. **Receive State**: Gets current conversation state
2. **Process Messages**: Uses the conversation chain
3. **Return Updates**: Provides new message to add to history

### Critical Implementation Details:
- **State Parameter**: Must accept the full state
- **Return Format**: Must return a dictionary with state updates
- **Message Handling**: Returns the LLM response as a new message

### Why This Works:
- **Automatic Accumulation**: `add_messages` appends the response
- **Context Preservation**: Full conversation history is maintained
- **Stateless Processing**: Node doesn't need to manage history

In [ ]:
def conversation_node(state: MyState) -> MyState:
    """Process conversation messages and generate responses.
    
    This node:
    1. Takes the current conversation state
    2. Passes all messages to the conversation chain
    3. Returns the LLM's response as a new message
    
    The add_messages annotation ensures the response is appended
    to the conversation history, not replacing it.
    
    Args:
        state: Current state with conversation messages
        
    Returns:
        State update with the chatbot's response message
    """
    response = conversation_chain().invoke({"messages": state["messages"]})
    return {"messages": response}

## 6. Memory Configuration

**MemorySaver** provides in-memory persistence for conversations.

### Memory Types in LangGraph:
1. **MemorySaver**: In-memory storage (this notebook)
2. **SqliteSaver**: Persistent SQLite storage (next notebook)
3. **RedisSaver**: Distributed Redis storage
4. **Custom Savers**: Implement your own storage backend

### Thread Configuration:
- **thread_id**: Unique identifier for each conversation
- **Isolation**: Different threads maintain separate conversations
- **Scalability**: Support multiple concurrent users

### When to Use MemorySaver:
- **Development**: Quick testing and prototyping
- **Single Session**: Conversations don't need to persist
- **Simple Applications**: Minimal setup requirements

### Limitations:
- **No Persistence**: Memory is lost when application restarts
- **Single Process**: Can't share across multiple instances
- **Memory Usage**: All conversations stored in RAM

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

# Initialize memory saver for conversation persistence
memory = MemorySaver()

# Configuration for thread management
config = {
    "configurable": {
        "thread_id": "user_1"  # Unique identifier for this conversation
    }
}

print(" Memory configuration ready")
print(f"Thread ID: {config['configurable']['thread_id']}")

## 7. Building the Graph

Our chatbot graph is intentionally simple:

```
START → conversation_node → END
```

### Graph Architecture:
- **Single Node**: All conversation logic in one place
- **Linear Flow**: No branching or conditional logic needed
- **Memory Integration**: Checkpointer handles state persistence

### Why This Simple Design Works:
- **Conversation Focus**: Single responsibility (chat responses)
- **Memory Handling**: Checkpointer manages conversation history
- **Extensibility**: Easy to add more nodes later

### Potential Extensions:
- **Intent Classification**: Route to different response types
- **Tool Integration**: Add function calling capabilities
- **Content Filtering**: Add safety and moderation checks

In [ ]:
# Build the conversation graph
graph = StateGraph(MyState)

# Add our single conversation node
graph.add_node("conversation", conversation_node)

# Set up the flow: START → conversation → END
graph.add_edge(START, "conversation")
graph.add_edge("conversation", END)

# Compile with memory checkpointer
app = graph.compile(checkpointer=memory)

print(" Graph compiled successfully with memory!")

## 8. Graph Visualization

Let's visualize our simple but powerful chatbot architecture:

In [ ]:
try:
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Visualization error: {e}")
    print("\nGraph Structure:")
    print("START → conversation_node → END")
    print("(with MemorySaver checkpointer for conversation persistence)")

## 9. Testing Memory Functionality

Let's test our chatbot's memory capabilities with a sequence of related messages.

### Test Scenario:
1. **Introduction**: User shares their name
2. **Memory Check**: Ask if the bot remembers
3. **Context Building**: Add more personal information
4. **Recall Test**: Verify the bot remembers multiple details

### What to Observe:
- **Context Retention**: Bot remembers previous messages
- **Natural Flow**: Responses build on conversation history
- **Personalization**: Bot uses remembered information appropriately

In [ ]:
from langchain_core.messages import HumanMessage

# Test 1: Introduction
print(" Test 1: User Introduction")
print("=" * 50)

question1 = "Hi! My name is Sarabjot and I'm learning LangGraph."
print(f"User: {question1}")

for event in app.stream({"messages": [HumanMessage(content=question1)]}, config=config):
    for value in event.values():
        print(f"Assistant: {value['messages'].content}")

print("\n" + "=" * 50)

In [ ]:
# Test 2: Memory Check
print(" Test 2: Memory Recall")
print("=" * 50)

question2 = "What is my name and what am I learning?"
print(f"User: {question2}")

for event in app.stream({"messages": [HumanMessage(content=question2)]}, config=config):
    for value in event.values():
        print(f"Assistant: {value['messages'].content}")

print("\n" + "=" * 50)

In [ ]:
# Test 3: Additional Context
print(" Test 3: Building Context")
print("=" * 50)

question3 = "I'm particularly interested in building chatbots with memory. Can you help me understand the key concepts?"
print(f"User: {question3}")

for event in app.stream({"messages": [HumanMessage(content=question3)]}, config=config):
    for value in event.values():
        print(f"Assistant: {value['messages'].content}")

print("\n" + "=" * 50)

In [ ]:
# Test 4: Comprehensive Recall
print("Test 4: Comprehensive Memory Test")
print("=" * 50)

question4 = "Can you summarize what you know about me and our conversation so far?"
print(f"User: {question4}")

for event in app.stream({"messages": [HumanMessage(content=question4)]}, config=config):
    for value in event.values():
        print(f"Assistant: {value['messages'].content}")

print("\n" + "=" * 50)

## 10. Multi-User Testing

Let's demonstrate how thread IDs enable separate conversations for different users.

### Thread Isolation:
- **Separate Contexts**: Each thread_id maintains its own conversation
- **No Cross-Talk**: Users can't see each other's conversations
- **Concurrent Support**: Multiple users can chat simultaneously

### Real-World Applications:
- **Multi-tenant SaaS**: Separate customer conversations
- **Customer Support**: Agent handles multiple tickets
- **Educational Platforms**: Individual student progress tracking

In [ ]:
# Create a second user configuration
config_user2 = {
    "configurable": {
        "thread_id": "user_2"
    }
}

print(" Multi-User Test: User 2 Introduction")
print("=" * 50)

user2_message = "Hello! I'm Alex and I'm new to AI development."
print(f"User 2: {user2_message}")

for event in app.stream({"messages": [HumanMessage(content=user2_message)]}, config_user2):
    for value in event.values():
        print(f"Assistant: {value['messages'].content}")

print("\n" + "=" * 50)

In [ ]:
# Test that User 1's context is still separate
print("🧪 Context Isolation Test: Back to User 1")
print("=" * 50)

isolation_test = "Do you remember my name?"
print(f"User 1: {isolation_test}")

for event in app.stream({"messages": [HumanMessage(content=isolation_test)]}, config):
    for value in event.values():
        print(f"Assistant: {value['messages'].content}")

print("\n User 1's context should be preserved (Sarabjot, LangGraph)")
print("ser 2's context should be separate (Alex, AI development)")

## Key Takeaways

### What I Learned:

1. **MessagesState**: Built-in conversation state management
2. **add_messages**: Automatic message accumulation for conversation flow
3. **MemorySaver**: In-memory conversation persistence
4. **Thread Management**: Separate conversation contexts per user
5. **Simple Architecture**: Minimal setup for maximum functionality
6. **Streaming**: Real-time conversation interaction

### When to Use This Pattern:

- **Conversational AI**: Any chatbot or assistant application
- **Customer Support**: Context-aware support interactions
- **Educational Tools**: Tutoring bots that remember student progress
- **Personal Assistants**: AI that learns user preferences over time

### Best Practices Demonstrated:

- **Type Safety**: Using TypedDict for state definition
- **Memory Integration**: Proper checkpointer configuration
- **Thread Isolation**: Separate contexts for different users
- **Simple Design**: Focus on core functionality first
- **Testing Strategy**: Verify memory and isolation capabilities

### Limitations of MemorySaver:

- **No Persistence**: Conversations lost on restart
- **Memory Usage**: All data stored in RAM
- **Single Process**: Can't share across multiple instances
- **No Backup**: Risk of data loss
